<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/11-TextClass/rnnTextClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tensorflow_datasets (tfds) data setler için lazım,
tensorflow kütüphanesi, modelin eğitiminde kullanılır.
matplotlib kütüphanesi grafik çizimleri için kullanılıyor.

In [22]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

Eğitim sırasında grafik çizme fonksiyonu Eğitim sırasında doğruluk ve kayıp grafiklerini çizmek için bir yardımcı fonksiyon tanımlanır

In [23]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

Veri seti, IMDB'nin büyük film inceleme verisetidir ve her yorum pozitif ya da negatif bir duygu içerir. Bu veriyi tfds.load ile indiriyoruz:

In [24]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

Yorum ve etiket örneği 1 beğenmek 0 ise beğnememek anlamına geliyor

In [25]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [26]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64


Bu pairleri  datasete çeviriyoruz

In [27]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [28]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b"Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings that made the earlier film such a delight. Miraglia puts more emphasis on the finer details of the plot this time around, and as a result it's the typical Giallo labyrinth, with characters all over the place and red herrings being thrown in every few minutes. This is a definite bonus for the film, however, as while it can get a little too confusing at times; there's always enough to hold the audience's interest and Miraglia's storytelling has improved since his earlier movie. The plot opens with a scene that sees two young girls fighting, before their grandfather explains to them the legend behind a rather lurid painting in their castle. The legend revolves around a woman called 'The

Ham metin verisini modelde kullanılabilir hale getirmek için TextVectorization katmanı kullanıyoruz. Bu katman, metni kelime dizilerine dönüştürür.TextVectorization: Metni sayısal verilere dönüştürür.

In [29]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [31]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

model

In [32]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,   1,  86, ...,   0,   0,   0],
       [142,   4, 196, ...,   0,   0,   0],
       [ 10,   1,   2, ...,   0,   0,   0]])

 [UNK] symbols, which typically represent unknown or unrecognized tokens in a model's vocabulary. This happens when certain words or characters in the original text aren't in the model's vocabulary, resulting in their replacement by the placeholder token.

In [33]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b"Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings that made the earlier film such a delight. Miraglia puts more emphasis on the finer details of the plot this time around, and as a result it's the typical Giallo labyrinth, with characters all over the place and red herrings being thrown in every few minutes. This is a definite bonus for the film, however, as while it can get a little too confusing at times; there's always enough to hold the audience's interest and Miraglia's storytelling has improved since his earlier movie. The plot opens with a scene that sees two young girls fighting, before their grandfather explains to them the legend behind a rather lurid painting in their castle. The legend revolves around a woman called 'T

In [34]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [37]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [42]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
input_text = tf.convert_to_tensor([sample_text])
predictions = model.predict(input_text)
print(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[0.00936007]


In [43]:
sample_text = ('The movie was bad. The animation and the graphics are poor '
               ' I would not recommend this movie.')

# Convert to tensor first
input_text = tf.convert_to_tensor([sample_text])
predictions = model.predict(input_text)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[0.00700458]]


In [45]:
# predict on a sample text with padding

padding = "the " * 2000
input_text = tf.convert_to_tensor([sample_text])
predictions = model.predict(input_text)
print(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[0.00700458]


In [46]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_2                 │ (1, None)                   │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (1, None, 64)               │          64,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (1, 128)                    │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1, 64)                     │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (1, 1)                      │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 138,369 (540.50 KB)

 Trainable params: 138,369 (540.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
137/391 ━━━━━━━━━━━━━━━━━━━━ 8:03 2s/step - accuracy: 0.5072 - loss: 0.6929

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
input_text = tf.convert_to_tensor([sample_text])
predictions = model.predict(input_text)
print(predictions[0])